# Import libraries, set options, connect to DB

In [1]:
# Configuration code for datawrangling
import pandas as pd
import os
import numpy as np
from datetime import datetime
from geocode import geocode
import mapToPoly
from mapToPoly import mapToPoly
pd.set_option('display.max_row', 30000)
import csv

# Configuration code in order to connect to the database
from sqlalchemy import create_engine, exists
from sqlalchemy.orm import sessionmaker
from database_setup import MixpanelMap, Base

passWord = os.environ['my_password']
# This commented out one was how I connected to the remote database
# DATABASE_URI = 'postgres://maxcarey:' + passWord + '@totago.cqfm37jhmjmk.ap-southeast-2.rds.amazonaws.com:5432/totago'
DATABASE_URI = 'postgres+psycopg2://maxcarey:' + passWord + '@localhost:5432/totago'
engine = create_engine(DATABASE_URI)

#engine = create_engine('sqlite:///totagoData.db')

# Bind the engine to the metadata of the Base class so that the
# declaratives can be accessed through a DBSession instance
Base.metadata.bind = engine

DBSession = sessionmaker(bind=engine)

session = DBSession()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


True
True


# Read in data as pandas data frame, selecting only certain fields

In [2]:
fields = ['distinct_id', 'numItinerariesReturned', 'departureDate', 'startFromLocation', 'selectedDestination_id', 'selectedDestination_name', 'time', 'user_id']

In [3]:
df = pd.read_csv('modified_iten.csv', usecols = fields, dtype={"selectedDestination_id" : "str"})

# Wrange field: destinationIDs

In [4]:
# Replace all of the NAs for destinationIDs with 0
df.selectedDestination_id.fillna(0, inplace = True)

## Remove the 2 cases where the string says null
## Great tutorial here: https://www.youtube.com/watch?v=2AFGPdNn4FM
df = df[df.selectedDestination_id != 'null']

## Convert destinationIDs column to an integer value
df['selectedDestination_id'] = df.selectedDestination_id.astype(int)

# Wrangle field: numItenerariesReturned

In [5]:
# Replace all of the NAs for numItinerariesReturned with 1
df.numItinerariesReturned.fillna(1, inplace = True)

# Convert from float to integer
df['numItinerariesReturned'] = df.numItinerariesReturned.astype(int)

# Select, only observatiosn where this field is greater than 0 (now that the NAs are gone)


# Wrangle Field: Destination Name

In [6]:
#Convert this field to an integer replacing all NA's with zero
# This gets rid of the trailing zeros
df.selectedDestination_name.fillna("", inplace = True)

# Wrangle Field: departureDate

In [7]:
#Convert destinationIDs column to an integer value
# It looks like there were some complex rows being held in here before, I thought that when df.dtypes returned object that
# meant string but apprently not
df['departureDate'] = df.departureDate.astype(str)

print("number of rows before removal of anamoulous departureDate cases")
print(len(df))

# IT looks like there are some cases where this field is blank, says nan, is in format 24503, or in format "masked" 
# We need to remove these cases from the data frame
# I can see that some blank rows are still printed out.
df = df[df.departureDate != '']
df = df[df.departureDate != 'nan']
df = df[df.departureDate != '24503']
df = df[df.departureDate != '[masked]']

print("number of rows after removal of anamoulous departureDate cases")
print(len(df))

# Create a function extractDate that extracts the first ten characters of an input string
def extractDate(dateString):
    extractedDate = dateString[0:10]
    if len(extractedDate) < 10:
        print(extractedDate)
    return extractedDate

''' Code to test if the extractDate function works

# Apply this function to create  a new column
df['departureDateFixed'] = df.departureDate.apply(extractDate)

cols = ['distinct_id', 'departureDate', 'departureDateFixed', 'numItinerariesReturned', 'selectedDestination_id', 'selectedDestination_name', 'startFromLocation']

df = df[cols]
'''

# Override departure date extracting all of the null time stamps
df['departureDate'] = df.departureDate.apply(extractDate)


# Convert departure date into a time object in pandas
#See here: https://stackoverflow.com/questions/26763344/convert-pandas-column-to-datetime
# Though this actually might not need to be done
#df['departureDate'] = df.departureDate.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))




#df['departureDate'] = datetime.strptime(df['departureDate'], '%Y-%m-%-d')  
#df['departureDate'] = pd.to_datetime(df['departureDate'], format = '%Y-%m-%-d')



number of rows before removal of anamoulous departureDate cases
35772
number of rows after removal of anamoulous departureDate cases
35764


# Wrangle Field: distinctID

In [8]:
# Create a coloumn that combines the unix time stamp with distinct_id so that we have a primary key for database
df["primary_key"] = df["distinct_id"] + "-" + df["time"].map(str)
vc = df.primary_key.value_counts()
unique_keys = df.primary_key.unique()

# Wrangle user_id field

In [9]:
df.dtypes

departureDate                object
distinct_id                  object
numItinerariesReturned        int64
selectedDestination_id        int64
selectedDestination_name     object
startFromLocation            object
time                          int64
user_id                     float64
primary_key                  object
dtype: object

In [10]:
# This gets rid of the railing zeros and all of the nas are just
# blank

df['user_id'] = df['user_id'].fillna(0).astype(np.int64)

#Convert to string to be consistent with other fields in database
df['user_id'] = df['user_id'].astype(str)


# Create a subset of the datle with sample method to test geocode and database entry logic

In [11]:
# Create a random sample of the database, these entries will be added to the database in the next section
sampleDf = df.tail(100)

# Output this random sample
sampleDf.head(len(sampleDf))

,departureDate,distinct_id,numItinerariesReturned,selectedDestination_id,selectedDestination_name,startFromLocation,time,user_id,primary_key
35672,2019-09-20,16cf382ea488c-0e30c36c6d71a78-7f642d12-4a574-1...,1,146,Quarry Rock,"Vancouver, British Columbia, Canada",1567428370,0,16cf382ea488c-0e30c36c6d71a78-7f642d12-4a574-1...
35673,2019-09-02,16cf3b722091c7-0750cd821508ec-5373e62-e1000-16...,1,127,Deer Lake,"49.232547,-123.18969140000002",1567431773,0,16cf3b722091c7-0750cd821508ec-5373e62-e1000-16...
35674,2019-09-20,16cf382ea488c-0e30c36c6d71a78-7f642d12-4a574-1...,1,146,Quarry Rock,"Vancouver, British Columbia, Canada",1567433035,0,16cf382ea488c-0e30c36c6d71a78-7f642d12-4a574-1...
35675,2019-09-02,16cf3d285db370-0ad546a9ee79fc8-4c312272-1aeaa0...,1,1146,Mailbox Peak,"Eastgate Park & Ride, 14200 SE Eastgate Way, W...",1567433615,827,16cf3d285db370-0ad546a9ee79fc8-4c312272-1aeaa0...
35676,2019-09-07,16cb12602c850b-0eac47430863c58-49183400-fa000-...,1,1148,Mount Teneriffe,"1010 East Alder Street, Seattle, Washington 98...",1567436675,827,16cb12602c850b-0eac47430863c58-49183400-fa000-...
35677,2019-09-02,16cf404f72d1f4-0563638dfac1e88-49183705-fa000-...,1,146,Quarry Rock,"1629 East 10th Avenue, Vancouver, British Colu...",1567436883,0,16cf404f72d1f4-0563638dfac1e88-49183705-fa000-...
35678,2019-09-07,16ceb6f65ae7f-04c75a0ddd752d-526e6332-38400-16...,1,9,Cougar Mountain Grand Traverse,"1226 5th Avenue North, Seattle, Washington 981...",1567437522,1,16ceb6f65ae7f-04c75a0ddd752d-526e6332-38400-16...
35679,2019-09-07,16ceb6f65ae7f-04c75a0ddd752d-526e6332-38400-16...,1,9,Cougar Mountain Grand Traverse,"1226 5th Avenue North, Seattle, Washington 981...",1567437644,1,16ceb6f65ae7f-04c75a0ddd752d-526e6332-38400-16...
35680,2019-09-07,9c20df17-ef03-4532-ad53-8e91ad7cec24,0,0,,"47.63061135,-122.34708349",1567438193,0,9c20df17-ef03-4532-ad53-8e91ad7cec24-1567438193
35681,2019-09-02,16cf418637c328-09ae566a56409a-c656943-4b6a2-16...,1,125,Cypress Falls,"13325 105th Ave, Surrey, British Columbia V3T ...",1567438394,0,16cf418637c328-09ae566a56409a-c656943-4b6a2-16...


## Read in the destination data to allow the possibility to pull the correct names

 


In [12]:
sampleDf.dtypes

departureDate               object
distinct_id                 object
numItinerariesReturned       int64
selectedDestination_id       int64
selectedDestination_name    object
startFromLocation           object
time                         int64
user_id                     object
primary_key                 object
dtype: object

# Loop through the rows in the dataframe, geocode, add entry to database

In [14]:
# Loop through the subsetted pandas data frame

# Uncomment the code below to loop through the the sample data frame
# for index, row in sampleDf.iterrows():

for index, row in sampleDf.iterrows():
  

    # Pull out the primary key into a variable
    testKey = row["primary_key"]
    
    # Check to see if that distinctID is in the data base
    # See this post: https://stackoverflow.com/questions/6587879/how-to-elegantly-check-the-existence-of-an-object-instance-variable-and-simultan?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa
    entryExists = session.query(exists().where(MixpanelMap.distinctkey==testKey)).scalar()

    # If the entry is not in the database
    if not entryExists:
    
        # Get the string to be geocoded
        locationToGeocode = row["startFromLocation"]

        # In the case that the desintaiton ID is zero, this means it was an NA
        # So don't even try to geocode
        
        if row["selectedDestination_id"] != 0:
        
            # Try to run the geocode function that returns a dictionary of information
            try:
                geocodeInfo = geocode(locationToGeocode)
                # If geocoding works, set valid to tre
                valid = True

            # If the geocode function doesn't work set valid to false
            except:
                valid = False
        
        # In the case that the selected Destination ID is 0 then set valid to false
        else:
            valid=False

        # If valid is true create a database entry with information from the dataframe, and the returned geocode informaiton
        if valid:
            
            # Sometimes, such as when a generic city is sent to the geocode() function a geometric center
            # is returned, this means there is no postal code
            
            # In this case, we can set the postalCode to none
            if not 'postalCode' in geocodeInfo:
                geocodeInfo['postalCode'] = "none"

            
            # Mapp the gps coordinates returned to the zip code polygons
            zipCodeInfo = mapToPoly(geocodeInfo['lat'], geocodeInfo['lng'], 'postal')
            
            
            if zipCodeInfo:
                zipCodeMapped = zipCodeInfo[0]
                region = zipCodeInfo[1]
            else:
                zipCodeMapped = 'outsideRegion'
                zipCodeMapped = 'outsideRegion'
            
            barrioInfo = mapToPoly(geocodeInfo['lat'], geocodeInfo['lng'], 'barrio')
            print(barrioInfo)
            
            if barrioInfo:
                barrioMapped = barrioInfo
            else:
                barrioMapped = 'outsideRegion'
                
            ## Get selected Destination Names
            # Pull the selected destination name
            selectedDestinationName = row["selectedDestination_name"]
            
            
            #if not selectedDestinationName:
            #    
            #    key = str(row["selectedDestination_id"])
            #    
            #    if key in destinations:
            #
            #        # Pull the data out from the dictionary that was created in the cell above
            #        newName = destinations[str(row["selectedDestination_id"])]['name']
        #
            #        # Add the new name to the new row
            #        selectedDestinationName  = newName
            #
            #    # In the case that there is destination that corresponds mark
            #    else:
            #    
            #        # TODO: CONSIDER CHANGING THE NAME OF THIS TO SOMETHING ELSE
            #        selectedDestinationName = "DELETED"
            #        # And overwrite valid to false at this point because there is no destination
            #        valid = False
            
            databaseEntry = MixpanelMap(distinctkey=row["primary_key"],
                                      numberitinerariesreturned=row["numItinerariesReturned"],
                                      selecteddestination_id=row["selectedDestination_id"],
                                      selecteddestination_name=selectedDestinationName,
                                      startfromlocation=row["startFromLocation"],
                                      departuredate=row["departureDate"],
                                      # Get data from python dictionary returned from geocode() function
                                      formatted_address=geocodeInfo['formatted_address'],
                                      lat=geocodeInfo['lat'],
                                      lng=geocodeInfo['lng'],
                                      postalcode=geocodeInfo['postalCode'],
                                      postalcodemapped=zipCodeMapped,
                                      barriomapped=barrioMapped,
                                      userid=row["user_id"],
                                      region=region,
                                      valid=valid)
        # If valid is false, just fill in the information that we have from the pandas data frame
        else:
            databaseEntry = MixpanelMap(distinctkey=row["primary_key"],
                                      numberitinerariesreturned=row["numItinerariesReturned"],
                                      selecteddestination_id=row["selectedDestination_id"],
                                      selecteddestination_name=row["selectedDestination_name"],
                                      startfromlocation=row["startFromLocation"],
                                      departuredate=row["departureDate"],
                                      userid=row["user_id"],
                                      valid=valid)

        # Add the the information to a database.    
        session.add(databaseEntry)
        session.commit()
    
    else:
        print("Entry already inside database")

{'formatted_address': 'Vancouver, BC, Canada', 'lat': 49.2827291, 'lng': -123.1207375}
CBD
{'formatted_address': '3843 SW Marine Dr, Vancouver, BC V6N 3Z7, Canada', 'lat': 49.23240939999999, 'lng': -123.1894639, 'postalCode': 'V6N 3Z7'}
DS
{'formatted_address': 'Vancouver, BC, Canada', 'lat': 49.2827291, 'lng': -123.1207375}
CBD
{'formatted_address': 'Eastgate Park and Ride, Bellevue, WA 98007, USA', 'lat': 47.5804416, 'lng': -122.1522951, 'postalCode': '98007'}
273579
{'formatted_address': '1010 E Alder St, Seattle, WA 98122, USA', 'lat': 47.6044949, 'lng': -122.3184252, 'postalCode': '98122'}
271869
{'formatted_address': '1629 E 10th Ave, Vancouver, BC V5N 1X6, Canada', 'lat': 49.2616355, 'lng': -123.0710023, 'postalCode': 'V5N 1X6'}
KC
{'formatted_address': '1226A 5th Ave N, Seattle, WA 98109, USA', 'lat': 47.63064130000001, 'lng': -122.3470575, 'postalCode': '98109'}
271856
{'formatted_address': '1226A 5th Ave N, Seattle, WA 98109, USA', 'lat': 47.63064130000001, 'lng': -122.347057

{'formatted_address': '112 Stewart St, Seattle, WA 98101, USA', 'lat': 47.61083989999999, 'lng': -122.3408456, 'postalCode': '98101'}
271849
{'formatted_address': '112 Stewart St, Seattle, WA 98101, USA', 'lat': 47.61083989999999, 'lng': -122.3408456, 'postalCode': '98101'}
271849
{'formatted_address': '1919 N 50th St, Seattle, WA 98103, USA', 'lat': 47.6648149, 'lng': -122.3343506, 'postalCode': '98103'}
252248
{'formatted_address': '4520 Beresford St, Burnaby, BC V5H 3C9, Canada', 'lat': 49.2244464, 'lng': -123.0019414, 'postalCode': 'V5H 3C9'}
None
{'formatted_address': 'Vancouver, BC V6H 3Y5, Canada', 'lat': 49.2701704, 'lng': -123.1356726, 'postalCode': 'V6H 3Y5'}
FAIR
{'formatted_address': '4111 80th Ave SE, Mercer Island, WA 98040, USA', 'lat': 47.57309, 'lng': -122.2326625, 'postalCode': '98040'}
None
{'formatted_address': '4111 80th Ave SE, Mercer Island, WA 98040, USA', 'lat': 47.57309, 'lng': -122.2326625, 'postalCode': '98040'}
None
{'formatted_address': '1018 Granville St,